## Imports

In [40]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [41]:
df = pd.read_csv('redfin_2023-11-03-12-18-03.csv')
BC_gf = gpd.read_file('countyboundary')

## Data Clean

In [42]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [43]:
df = df.dropna(subset=['SOLD DATE'])

In [44]:
# Define list of desired months (excluding current month)
desired_months = ['October']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [45]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    851
Name: count, dtype: int64
-------
$/SQUARE FEET
False    841
True      10
Name: count, dtype: int64
-------
YEAR BUILT
False    851
Name: count, dtype: int64
-------


In [46]:
# sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
# second_newest_building = sorted_df.iloc[2]
# print(second_newest_building['URL'])

In [47]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [48]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [49]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
647,PAST SALE,October-11-2023,Condo/Co-op,2880 N Oakland Frst #114,Oakland Park,FL,33309.0,268.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Oakland-Park/2880-N-...,MARMLS,A11411685,N,Y,26.171611,-80.181625
211,PAST SALE,October-25-2023,Condo/Co-op,4740 NW 21st St #201,Lauderhill,FL,33313.0,13000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4740-NW-2...,MARMLS,A11424052,N,Y,26.152403,-80.213786
742,PAST SALE,October-10-2023,Condo/Co-op,13000 SW 15th Ct Unit 303U,Pembroke Pines,FL,33027.0,25000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/13000...,MARMLS,A11393898,N,Y,25.994581,-80.320547
43,PAST SALE,October-10-2023,Condo/Co-op,3160 Holiday Springs Blvd #104,Margate,FL,33063.0,67000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3160-Holiday...,Beaches MLS,F10398276,N,Y,26.265849,-80.233308
503,PAST SALE,October-27-2023,Condo/Co-op,1330 NW 43rd Ave #207,Lauderhill,FL,33313.0,78000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1330-NW-4...,Beaches MLS,F10396742,N,Y,26.143807,-80.206741
723,PAST SALE,October-5-2023,Condo/Co-op,3190 Holiday Springs Blvd Unit 4-202,Margate,FL,33063.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3190-Holiday...,MARMLS,A11386200,N,Y,26.267328,-80.233569
161,PAST SALE,October-19-2023,Condo/Co-op,115 Upminster E #115,Deerfield Beach,FL,33442.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/115-...,Beaches MLS,F10397564,N,Y,26.313249,-80.129323
235,PAST SALE,October-11-2023,Condo/Co-op,9061 Sunrise Lakes Blvd #201,Sunrise,FL,33322.0,82000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9061-Sunrise...,Beaches MLS,F10388685,N,Y,26.163488,-80.272944
525,PAST SALE,October-11-2023,Condo/Co-op,2793 NW 104th Ave #105,Sunrise,FL,33322.0,82500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2793-NW-104t...,Beaches MLS,RX-10899152,N,Y,26.160121,-80.288192
407,PAST SALE,October-27-2023,Condo/Co-op,2800 Somerset Dr Unit 405J,Lauderdale Lakes,FL,33311.0,84000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/280...,MARMLS,A11434188,N,Y,26.160830,-80.188993


In [56]:
print(df_filtered['URL'].iloc[742])

https://www.redfin.com/FL/Pembroke-Pines/13000-SW-15th-Ct-33027/unit-303U/home/186156594


In [57]:
# Correct the prices, if needed
df_filtered.at[647,'PRICE']=(268000)
df_filtered.at[211,'PRICE']=(130000)
df_filtered.at[742,'PRICE']=(250000)

In [58]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [59]:
# # Corrections, if needed
df_filtered.at[647,'$/SQUARE FEET']=(268000/1038)
df_filtered.at[211,'$/SQUARE FEET']=(130000/928)
df_filtered.at[742,'$/SQUARE FEET']=(250000/1082)

In [60]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
43,67000.0,3160 Holiday Springs Blvd #104,Margate,92.0
558,99999.0,4050 NW 42nd Ave #415,Lauderdale Lakes,92.0
23,107500.0,2504 Antigua Ter Unit G3,Coconut Creek,93.0
597,90000.0,1380 NW 43rd Ter #301,Lauderhill,95.0
575,85000.0,1480 NW 80th Ave #105,Margate,100.0
813,107000.0,3821 Environ Blvd #407,Lauderhill,101.0
56,97000.0,4851 NW 21st St #211,Lauderhill,105.0
260,165000.0,3301 Spanish Moss Ter #607,Lauderhill,106.0
503,78000.0,1330 NW 43rd Ave #207,Lauderhill,107.0
506,132000.0,4401 NW 41st St #406,Lauderdale Lakes,110.0


In [61]:
print(df_filtered.URL.iloc[43])

https://www.redfin.com/FL/Margate/3160-Holiday-Springs-Blvd-33063/unit-104/home/187327340


In [19]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [62]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [63]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [64]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [65]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [66]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [67]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"October 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(BC_gf,tooltip='Broward County',name='Broward County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [68]:
m.save('index.html')

## Summary Info

In [69]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [70]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [71]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [72]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Pompano-Beach/1100-S-Ocean-Blvd-33062/unit-D16/home/41699667


In [73]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Auberge Beach Residences, 2200 N Ocean Blvd Unit N1501 Fort Lauderdale | Price $6,700,000 | $1,958 psf | Year built: 2018
Least Expensive
Holiday Springs, 3160 Holiday Springs Blvd #104 Margate | Price $67,000 | $92 psf | Year built: 1974
Highest Price Per Square Foot
Auberge Beach Residences, 2200 N Ocean Blvd Unit N1501 Fort Lauderdale | Price $6,700,000 | $1,958 psf | Year built: 2018
Lowest Price Per Square Foot
Topaz North, 4050 NW 42nd Ave #415 Lauderdale Lakes | Price $99,999 | $92 psf | Year built: 1974
Newest
METROPICA NORTH TOWER ONE, 2000 Metropica Way #2403 Sunrise | Price $768,900 | $652 psf | Year built: 2020
Oldest
Pompano Surf Club, 1100 S Ocean Blvd Unit D16 Pompano Beach | Price $1,800,000 | $1,417 psf | Year built: 1955


## Time on Market Calculator

In [84]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Sunrise/2000-Metropica-Way-33323/unit-2403/home/145208285


In [80]:
from datetime import datetime, timedelta

date1 = datetime(2023, 9, 2) ## List (Earlier) date
date2 = datetime(2023, 10, 10) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

38


## Map URL Snagger

In [75]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [76]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_oct_2023


## Get Summary Data

In [77]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 851
--------
Total sale price: $277,195,728
--------
Median sale price: $240,000
--------
Max sale price: $6,700,000
--------
Min sale price: $67,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,958
--------
Min price per square foot: $92
--------
Median price per square foot: $237
------------------------------------------------
CONDO AGES
Newest building: 2020.0
----------
Oldest building: 1955.0
----------
Average building age: 1980.4935370152762
